# 2. Black-Scholes Baseline

This notebook implements and evaluates the standard Black-Scholes model as the baseline for option pricing. Despite its simplifying assumptions, the Black-Scholes model remains a foundational benchmark for option pricing. In this notebook, we use it to evaluate market prices and establish a baseline before introducing more flexible models like Heston.

## Goals
- Implement analytical formulas for Black-Scholes pricing of **vanilla** European options (calls and puts).
- Compute theoretical option prices using real market data.
- Calculate key **Greeks**: Delta, Gamma, Vega, Theta, and Rho.
- Evaluate the model's accuracy by comparing theoretical prices with observed market mid prices.
- Visualize volatility surfaces and model pricing errors.

## Structure
1. **Environment & Data**: Load cleaned options dataset and libraries.
2. **Black-Scholes Functions**: Pricing formula and Greeks.
3. **Application to Market Data**: Compute theoretical prices and compare with mid prices.
4. **Visualizations**: Price surfaces, Greeks, and model error heatmaps.
5. **Conclusion**: Performance analysis and motivation for moving beyond Black-Scholes.

*Note* This introduction is done mainly in a very theoretical framwork. If you want to skip the derivation, go directly to section 3.

## 1. Environment & Data

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
data_path  = '../data/options_cleaned.csv'
options_df = pd.read_csv(data_path)
options_df.head()

,strike,bid,ask,lastPrice,openInterest,impliedVolatility,Type,T,mid,spread,risk_free_rate,Underlying_price
0,1400.0,4971.1,4990.3,4868.23,1432.0,8.709477,call,0.0,4980.70,19.2,0.0433,6374.890137
1,4000.0,2376.7,2383.4,2292.49,108.0,2.858401,call,0.0,2380.05,6.7,0.0433,6374.890137
2,4400.0,1976.4,1982.7,2001.24,163.0,2.298466,call,0.0,1979.55,6.3,0.0433,6374.890137
3,5000.0,1377.1,1381.7,1383.13,136.0,1.578127,call,0.0,1379.40,4.6,0.0433,6374.890137
4,5200.0,1177.2,1181.4,1183.17,2721.0,1.350345,call,0.0,1179.30,4.2,0.0433,6374.890137


## 2. Black-Scholes model: definition and theoretical solution.

The Black-Scholes model is a mathematical model used in the description of financial markets that contains derivative investment instruments. It was initially proposed by Fischer Black and Myron Scholes in 1973 in the article "The Pricing of Options and Corporate Liabilities".

### 2.1. Model assumptions.

The model make the following assumptions about the assets:
- Risk-free rate: The rate of return on the riskless asset is constant.
- Random walk: The instantaneous log return of the stock price is an infinitesimal random walk with constant drift and volatility.
- The stock does not pay a dividend.

On the other hand, the market asumpitons that are made are:
- No arbitrage opportunity (no way to make a riskless profit in excess of the risk-free rate).
- Ability to borrow and lend any amount of cash at the riskless rate.
- Ability to buy and sell any amount, even fractional, of the stock.
- The above transactions do not incur any fees or costs.

### 2.2. Black-Scholes equation derivation.

*Note*: All pricing and calibration in this notebook is done under the risk-neutral measure $\mathbb{Q}$, as standard in quantitative finance. The implied volatilities and risk-free rate are consistent with this framework.

In practical terms, this assumtions can be formalize by describing the evolution of the underlying asset as a stochastic process whose dynamical equation is a geometric brownian motion:

$$
dS_t = r_f \cdot S_t \cdot dt + \sigma\cdot S_t \cdot dW^{\mathbb{Q}}
$$

where $r_f$ is the risk-free rate, the $\sigma$ is the stock volatility and $dW^{\mathbb{Q}}$ is a diferential Wiener process, which constitues the only source of randomness in the model.

Now, by Ito's lemma, we can compute the evolution of any function $V(S_t, t)$ that depends on $S_t$. The differential equation obtained is:

$$
dV = \left(\frac{\partial V}{\partial t} +  r_f S_t \,\frac{\partial V}{\partial S_t} + \frac{1}{2}\,\sigma^2 S^2 \,\frac{\partial^2 V}{\partial S^2}\right)\cdot dt + \sigma S_t\,\frac{\partial V}{\partial S_t} \,dW^{\mathbb{Q}}
$$

The last financial instrument that we are going to take is a portfolio $\Pi$ consisting of a single option with value $V(S_t, t)$ and the underlying $S_t$. The value of the portfolio is given by:

$$
\Pi = V(S_t, t) + N_s \cdot S_t
$$

and its evolution by:

$$
d\Pi = \left[ \frac{\partial V}{\partial t} + r_f S_t\,\frac{\partial V}{\partial S} + \frac{1}{2}\,\sigma^2 S_t^2\,\frac{\partial^2 V}{\partial S^2} + N_s r_f S_t \right] \,dt + \sigma\,S_t\cdot\left[\frac{\partial V}{\partial S} + N_s  \right] \,dW^{\mathbb{Q}}
$$

Note now that we can remove the dependence of $\Pi$ with the underlying noise $dW^{\mathbb{Q}}$ by selecting $N_s = -\frac{\partial V}{\partial S}$. This is called **delta-hedge** and is fundamental in portfolio managment as, according to Black-Scholes model, the portfolio is now riskless.
$$
d\Pi =  \left[ \frac{\partial V}{\partial t} + \frac{1}{2}\,\sigma^2 S_t^2\,\frac{\partial^2 V}{\partial S^2}  \right] \,dt 
$$

The rate of return on this portfolio must be equal to the rate of return on any other riskless instrument; otherwise, there would be opportunities for arbitrage. As a consequence:

$$
d\Pi = r_f \Pi dt = r_f\left[V(S_t, t) - \frac{\partial V}{\partial S} \cdot S_t  \right]\, dt
$$

By combining both formulas, we arrive at the Black–Scholes partial differential equation:

$$
\frac{\partial V}{\partial t} + r_f S_t \frac{\partial V}{\partial S} + \frac{1}{2}\sigma^2 S^2 \frac{\partial S^2}{\partial S^2} = rV
$$

### 2.3. Solution for the European vanilla option

The previous Black-Scholes equation has a known solution in the case of the European options. For the call options, the solution is:

$$
C(S, T, \sigma, r_f) = N(d_1) S - N(d_2)K e^{-r_f\,T}
$$

where $N(x)$ is the normal cumulative function, $K$ is the striking price, $T$ the time to maturity and
$$
d_1 = \frac{1}{\sigma \sqrt{T}}\,\left[\ln\left(\frac{S}{K}\right) +  \left(r_f + \frac{\sigma^2}{2}\right) T\right]
$$
$$
d_2 = d_1 - \sigma \sqrt{N}
$$

The price of the put option can be computed from this using the put-call parity:

$$
P(S, T, \sigma, r_f)  = K e^{-r_f T} - S + C(S, T, \sigma, r_f)
$$

### 2.4. Geeks options.

The Black-Scholes solution also gives the so called **option greeks** value. These measure the sensitivity of the value of a derivative product to changes in parameter values while holding the other parameters fixed. The most important are the following:
- Delta Greek.
$$
\Delta_{\text{call}}  = \frac{\partial V}{\partial S} = N(d_1) \quad \Delta_{\text{put}}  = \frac{\partial V}{\partial S} = N(d_1) - 1
$$

- Gamma Greek.
$$
\Gamma_{\text{call}} , \Gamma_{put} = \frac{\partial^2 V}{\partial S^2} = \frac{N'(d_1)}{S\sigma \sqrt{T}}
$$

There are additionally other three that should be taken into account:
- Vega Greek
$$
\nu_{\text{call}},\, \nu_{\text{put}} = \frac{\partial V}{\sigma} = S N'(d_1) \sqrt{T}
$$

- Theta Greek.
$$
\Theta_{\text{call}} = \frac{\partial V}{\partial t} = - \frac{S N'(d_1) \sigma}{2\sqrt{T}} - rK e^{-r T}N(d_2)  \quad \Theta_{\text{put}} = \frac{\partial V}{\partial t} = - \frac{S N'(d_1) \sigma}{2\sqrt{T}} - rK e^{-r T}(N(d_2) - 1)
$$

- Rho Greek.
$$
\rho_{\text{call}} = \frac{\partial V}{\partial r} = K T e^{-r_f T} N(d_2) \quad \quad \rho_{\text{put}} =\frac{\partial V}{\partial r} = K T e^{-r_f T} (N(d_2) - 1)
$$

### 2.5. Code implementation

Although these expressions are complex, they can be easily computed with the `scipy.stats` library. The code is written in the `Black_scholes.py` script.

In [5]:
import sys
import os

script_path = os.path.abspath(os.path.join("..", "Scripts"))
if script_path not in sys.path:
    sys.path.append(script_path)

import Black_scholes 